In [23]:
import numpy as np # for mathematics
import pandas as pd # for processing of data
sh = pd.read_csv('spam.csv',encoding='latin-1')

sh = sh.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
sh = sh.rename(columns={"v1":'label', "v2":'text'})
print(sh.head())
tags = sh["label"]
texts = sh["message"]

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [25]:
num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(texts,tags, test_size = 0.3)
mat_texts_tr = tok.texts_to_matrix(x_train,mode='count')
mat_texts_tst = tok.texts_to_matrix(x_test,mode='count')

max_len = 100
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test)

In [27]:
def get_simple_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(num_max,)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return (model)

In [28]:
def check_model(model,xtr,ytr,xts,yts):
    model.fit(xtr,ytr,batch_size=32,epochs=10,verbose=1,validation_split=0.3)
    print(' ')
    model.evaluate(xts,yts)

In [29]:
def predict_model(model, x_test, y_test):
    prediction = model.predict(x_test)
    prediction = np.ceil(prediction)
    accuracy = accuracy_score(prediction, y_test)
    return accuracy

def predict_model_output(model, x_test):
    prediction = model.predict(x_test)
    prediction = np.ceil(prediction)
    return prediction

In [30]:
# for testing accuracy
m = get_simple_model()
check_model(m,mat_texts_tr,y_train,mat_texts_tst,y_test)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               512512    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 644,097
Trainable params: 644,097
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
86/86 [==============================

In [32]:
spam_test = pd.read_csv('output_spam.csv', encoding='latin-1')
mat_texts_out = tok.texts_to_matrix(spam_test["text"],mode='count')
prediction_final = predict_model_output(m, mat_texts_out)
prediction_test_out = pd.DataFrame(prediction_final, columns=['Text_type']).to_csv("output.csv")